Sprawozdanie 

Use Case
W tym projekcie użyto datasetu związanego z parametrami i zarobkami koszykarzy w lidze NBA.
Wyniki przechowywane są w kolumnie salary.
W przypadku tego rodzaju danych użyto modelu regresji do przewidywania zarobków gracza na podstawie pozostałych parametrów.

Kroki:
1. Stworzenie resource group i zasobu Machine Learning.
2. Stworzenie Compute Instance i Cluster Instance.
3. Załadowanie danych - ustawienie kolumny target jako kolumny predykcji
4. Uruchomienie Automated ML 
Pierwszym krokiem było załadowanie zestawu danych w formacji .csv i skonfigurowanie ich migracji (m. in. rozpoznanie nazw kolumn, ustalenie która kolumna jest targetem klasyfikacji) do chmury.
Plik z datasetem: https://www.kaggle.com/isaienkov/nba2k20-player-dataset/tasks?taskId=1925
Drugim przeprowadzenie operacji AutoML, która pozwoliła na przegląd skuteczności modeli.


Jupyter notebook

Azure ML Workspace

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

# Get Workspace defined in by default config.json file
ws = Workspace.from_config()

Ładowanie danych i wyświetlenie pierwszych pięciu kolumn wraz z statystykami danych liczbowych parametrów. 

In [3]:

# Load Data
aml_dataset = ws.datasets['nba2020']
full_df = aml_dataset.to_pandas_dataframe()
full_df.head(5)


,full_name,rating,team,position,b_day,height,weight,salary,country,draft_year,draft_round,draft_peak,college
0,LeBron James,97,Los Angeles Lakers,F,1984-12-30,6-9 / 2.06,250 lbs. / 113.4 kg.,37436858,USA,2003,1,1,None
1,Kawhi Leonard,97,Los Angeles Clippers,F,1991-06-29,6-7 / 2.01,225 lbs. / 102.1 kg.,32742000,USA,2011,1,15,San Diego State
2,Giannis Antetokounmpo,96,Milwaukee Bucks,F-G,1994-12-06,6-11 / 2.11,242 lbs. / 109.8 kg.,25842697,Greece,2013,1,15,None
3,Kevin Durant,96,Brooklyn Nets,F,1988-09-29,6-10 / 2.08,230 lbs. / 104.3 kg.,37199000,USA,2007,1,2,Texas
4,James Harden,96,Houston Rockets,G,1989-08-26,6-5 / 1.96,220 lbs. / 99.8 kg.,38199000,USA,2009,1,3,Arizona State


In [4]:
full_df.describe()

,rating,salary,draft_year
count,429.000000,4.290000e+02,429.000000
mean,76.372960,8.530080e+06,2014.093240
std,5.657748,9.218059e+06,3.937678
min,67.000000,5.000000e+04,2001.000000
25%,72.000000,2.000000e+06,2012.000000
50%,75.000000,4.380120e+06,2015.000000
75%,79.000000,1.220000e+07,2017.000000
max,97.000000,4.023176e+07,2019.000000


Podział dane na uczące i testujące.

In [5]:
train_dataset, test_dataset = aml_dataset.random_split(0.9, seed=1)

train_dataset_df = train_dataset.to_pandas_dataframe()
test_dataset_df = test_dataset.to_pandas_dataframe()

print(train_dataset_df.describe())

           rating        salary   draft_year
count  383.000000  3.830000e+02   383.000000
mean    76.543081  8.546792e+06  2014.054830
std      5.758908  9.270938e+06     4.014323
min     67.000000  7.956800e+04  2001.000000
25%     72.000000  2.028594e+06  2011.000000
50%     75.000000  4.380120e+06  2015.000000
75%     79.000000  1.227284e+07  2017.000000
max     97.000000  4.023176e+07  2019.000000


Listowanie danych konfigu danych związanych z Compute instances, compute clasters itd.

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ComputeTarget.list(ws)

[{
   "id": "/subscriptions/ba3b040d-bcca-454d-ba76-707db0d50dff/resourceGroups/lab_3/providers/Microsoft.MachineLearningServices/workspaces/lab_3_ml/computes/CI-ML",
   "name": "CI-ML",
   "location": "westeurope",
   "tags": null,
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "westeurope",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "STANDARD_DS11_V2",
       "applications": [
         {
           "displayName": "Jupyter",
           "endpointUri": "https://ci-ml.westeurope.instances.azureml.ms"
         },
         {
           "displayName": "Jupyter Lab",
           "endpointUri": "https://ci-ml.westeurope.instances.azureml.ms/lab"
         },
         {
           "displayName": "RStudio",
           "endpointUri": "https://ci-ml-8787.westeurope.instances.azureml.ms"
         }
       ],
       "connectivityEndpoints": {
       

Łączenie ze zdalnym AML Compute

In [7]:
amlcompute_cluster_name = "CC-AutoML"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     # Get existing cluster
     aml_remote_compute = cts[amlcompute_cluster_name]
        
print('Checking cluster status...')
aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)

Found existing training cluster.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
aml_remote_compute.get_status().serialize()

{'currentNodeCount': 1,
 'targetNodeCount': 1,
 'nodeStateCounts': {'preparingNodeCount': 0,
  'runningNodeCount': 1,
  'idleNodeCount': 0,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Steady',
 'allocationStateTransitionTime': '2021-01-31T20:50:54.217000+00:00',
 'errors': None,
 'creationTime': '2021-01-31T17:35:45.431912+00:00',
 'modifiedTime': '2021-01-31T17:36:01.293665+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 1,
  'maxNodeCount': 2,
  'nodeIdleTimeBeforeScaleDown': 'PT120S'},
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_DS12_V2'}

Uzyskanie dostępnych metod wyliczenia regresji, kod pomocniczy aby uzystać nazwę potrzebną w następnym klastrze kodu.

In [14]:
from azureml.train import automl
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('regression')

['normalized_root_mean_squared_error',
 'r2_score',
 'normalized_mean_absolute_error',
 'spearman_correlation']

Definiowanie AutoML ustawień eksperymentu, dane wykorzystane m.in z kodu powyżej, jak i określenie jaka kolumna jest targetem, maksymalny czas obliczeń.

In [9]:
import logging
import os

from azureml.train.automl import AutoMLConfig

project_folder = './projekt_lab_3'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='regression',
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_minutes=40,                            
                             training_data=train_dataset,
                             label_column_name="salary",
                             n_cross_validations=5,                                                 
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder
                             )

Przeprowadzenie eksperymentu oraz kod obserwujący czas wykonywanych obliczeń.

In [10]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "ipynb-experiment-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

ipynb-experiment-01-31-2021-23
Running on remote.
No run_configuration provided, running on CC-AutoML with default configuration
Running on remote compute: CC-AutoML
Parent Run ID: AutoML_2be82eef-3aa8-4e38-bccb-b32fef140c2c

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      

W tym miejscu napotkano błąd w którym notebook nie potrafi poprawnie uruchomić widgetu. Edytor sugeruje, że widgets nie jest częścią azureml, 
co (po sprawdzeniu) jest niezgodne z dokumentacją azura (przeprowadzano również akutalizacje paczek).

In [11]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Mierzenie czas wykonywania Parenta niezbędnego do całego procesu AutoML. 

In [12]:
import time
import datetime as dt

run_details = run.get_details()

end_time_utc_str = run_details['endTimeUtc'].split(".")[0]
start_time_utc_str = run_details['startTimeUtc'].split(".")[0]
timestamp_end = time.mktime(datetime.strptime(end_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())
timestamp_start = time.mktime(datetime.strptime(start_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())

parent_run_time = timestamp_end - timestamp_start
print('Run Timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (parent_run_time))

Run Timing: --- 2463.0 seconds needed for running the whole Remote AutoML Experiment ---


Otrzymanie najlepszego modelu

In [19]:
best_run, fitted_model = run.get_output(29)
print(best_run)
print(fitted_model)

ImportError: cannot import name 'UnhashableValueInColumn'

Wyekstrahowanie kolumny z wynikami i konwersja do tablicy NumPi do procesu przewidywania.

In [21]:
import pandas as pd

#Remove Label/y column
if 'Attrition' in test_dataset_df.columns:
    y_test_df = test_dataset_df.pop('salary')

x_test_df = test_dataset_df

Proces przewidywania niemożliwy do osiągnięcia poprzez błędy przedstawione powyżej.

In [22]:
# Try the best model
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])

NameError: name 'fitted_model' is not defined